<a href="https://colab.research.google.com/github/keshavkundra/Machine-learing/blob/main/KESHAVKUNDRA(ASSINMENT2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 1. Install required packages
!pip install -q kaggle pandas

import os
import pandas as pd

# 2. Upload your kaggle.json API key
from google.colab import files
print("Please upload your kaggle.json file:")
files.upload()  # This will prompt you to select the file

# 3. Move kaggle.json to the proper directory and set permissions
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# 4. Download the dataset using the Kaggle API
!kaggle datasets download -d jahias/microsoft-adventure-works-cycles-customer-data -p /content --unzip

# 5. Load the dataset
import glob
csv_files = glob.glob('/content/*.csv')
print("Found CSV file(s):", csv_files)

# Adjust if filename differs
df = pd.read_csv(csv_files[0], low_memory=False)
print("Loaded dataset shape:", df.shape)


Please upload your kaggle.json file:


Saving AWCustomers.csv to AWCustomers.csv
Saving AWSales.csv to AWSales.csv
Saving AWTest-Classification.csv to AWTest-Classification.csv
Saving AWTest-Regression.csv to AWTest-Regression.csv
mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~

Part I: Based on Feature Selection, Cleaning, and Preprocessing to Construct an Input from Data
Source
(a) Examine the values of each attribute and Select a set of attributes only that would affect to predict
future bike buyers to create your input for data mining algorithms. Remove all the unnecessary
attributes. (Select features just by analysis).
(b) Create a new Data Frame with the selected attributes only.
(c) Determine a Data value type (Discrete, or Continuous, then Nominal, Ordinal, Interval, Ratio) of
each attribute in your selection to identify preprocessing tasks to create input for your data mining.


In [3]:
# 1. Import libraries
import pandas as pd
import numpy as np

# 2. Load the dataset from the uploaded CSV
df = pd.read_csv('AWCustomers.csv', low_memory=False)
print("Original dataset shape:", df.shape)
print("Columns:", df.columns.tolist())

# 3. Quick initial inspection
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
df.info()

# 4. Select features relevant for predicting BikeBuyer
selected_cols = [
    'BirthDate',           # to derive Age
    'YearlyIncome',
    'Education',
    'Occupation',
    'Gender',
    'MaritalStatus',
    'HomeOwnerFlag',
    'NumberCarsOwned',
    'NumberChildrenAtHome',
    'TotalChildren',
    'CommuteDistance',     # if present
    'Region',              # if present
    'StateProvinceName',   # if present
    'BikeBuyer'            # target variable
]
available = [col for col in selected_cols if col in df.columns]
print("\nSelected (available) columns:", available)

# 5. Subset dataset
df_sel = df[available].copy()
print("\nSubset shape:", df_sel.shape)
print(df_sel.head())

# 6. Determine data types and measurement scales for each selected attribute
dtype_map = {}
for col in df_sel.columns:
    s = df_sel[col]
    if pd.api.types.is_numeric_dtype(s):
        unique = s.nunique()
        typ = 'Continuous' if unique > 15 else 'Discrete'
        scale = 'Ratio' if (s.dropna() >= 0).all() and s.min() == 0 else 'Interval'
    else:
        if col == 'Education':
            typ, scale = 'Categorical', 'Ordinal'
        elif col in ['Gender', 'MaritalStatus', 'Occupation', 'CommuteDistance', 'Region', 'StateProvinceName']:
            typ, scale = 'Categorical', 'Nominal'
        elif col in ['HomeOwnerFlag', 'BikeBuyer']:
            typ, scale = 'Binary', 'Nominal'
        else:
            typ, scale = 'Categorical', 'Nominal'
    dtype_map[col] = f"{typ}, {scale}"

print("\nData Type & Scale for each attribute:")
for attr, desc in dtype_map.items():
    print(f"- {attr}: {desc}")


Original dataset shape: (18361, 24)
Columns: ['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate', 'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'LastUpdated']

First 5 rows:
   CustomerID Title FirstName MiddleName  LastName Suffix  \
0       21173   NaN      Chad          C      Yuan    NaN   
1       13249   NaN      Ryan        NaN     Perry    NaN   
2       29350   NaN     Julia        NaN  Thompson    NaN   
3       13503   NaN  Theodore        NaN     Gomez    NaN   
4       22803   NaN  Marshall          J      Shan    NaN   

             AddressLine1 AddressLine2         City    StateProvinceName  ...  \
0      7090 C. Mount Hood          NaN   Wollongong      New South Wales  ...   
1     3651 Willow Lake Rd          NaN      Shawnee 

Part II: Data Preprocessing and Transformation
Depending on the data type of each attribute, transform each object from your preprocessed data.
Use all the data rows (~= 18000 rows) with the selected features as input to apply all the tasks below, do
not perform each task on the smaller data set that you got from your random sampling result.
(a) Handling Null values
(b) Normalization
(c) Discretization (Binning) on Continuous attributes or Categorical Attributes with too many different
values
(d) Standardization/Normalization
(e) Binarization (One Hot Encoding)

In [10]:
import pandas as pd

df = pd.read_csv('AWCustomers.csv', low_memory=False)
print("Columns in AWCustomers.csv:")
print(df.columns.tolist())



Columns in AWCustomers.csv:
['CustomerID', 'Title', 'FirstName', 'MiddleName', 'LastName', 'Suffix', 'AddressLine1', 'AddressLine2', 'City', 'StateProvinceName', 'CountryRegionName', 'PostalCode', 'PhoneNumber', 'BirthDate', 'Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome', 'LastUpdated']


In [12]:
from sklearn.preprocessing import OneHotEncoder

# Updated encoder initialization
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

# Fit and transform your categorical features
X_ohe = pd.DataFrame(
    ohe.fit_transform(df[cat_cols]),
    columns=ohe.get_feature_names_out(cat_cols)
)


In [13]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, KBinsDiscretizer, OneHotEncoder

# Load data and derive Age
df = pd.read_csv('AWCustomers.csv', low_memory=False)
df['Age'] = pd.to_datetime(df['BirthDate'], errors='coerce').pipe(
    lambda x: (pd.Timestamp.today() - x).dt.days // 365
)
df.drop(columns=['BirthDate'], inplace=True)

# Select numeric and categorical columns
num_cols = [c for c in ['Age', 'YearlyIncome', 'NumberCarsOwned', 'NumberChildrenAtHome', 'TotalChildren'] if c in df]
cat_cols = [c for c in ['Education', 'Occupation', 'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'CommuteDistance', 'Region', 'StateProvinceName'] if c in df]

# Handle missing values
df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])
df[cat_cols] = SimpleImputer(strategy='most_frequent').fit_transform(df[cat_cols])

# Min-Max scaling
df_minmax = pd.DataFrame(
    MinMaxScaler().fit_transform(df[num_cols]),
    columns=[f"{c}_minmax" for c in num_cols]
)

# Quantile binning
bin_feats = [c for c in ['Age', 'YearlyIncome'] if c in df]
df_binned = pd.DataFrame(
    KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
        .fit_transform(df[bin_feats]).astype(int),
    columns=[f"{c}_binned" for c in bin_feats]
) if bin_feats else pd.DataFrame()

# Z-score standardization
df_std = pd.DataFrame(
    StandardScaler().fit_transform(df[num_cols]),
    columns=[f"{c}_std" for c in num_cols]
)

# One-Hot Encoding (with updated parameter)
ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
df_ohe = pd.DataFrame(
    ohe.fit_transform(df[cat_cols]),
    columns=ohe.get_feature_names_out(cat_cols)
)

# Combine all transformed features
df_transformed = pd.concat([df_minmax, df_binned, df_std, df_ohe], axis=1)
print("Transformed DataFrame shape:", df_transformed.shape)
df_transformed.head()


Transformed DataFrame shape: (18361, 76)


,Age_minmax,YearlyIncome_minmax,NumberCarsOwned_minmax,NumberChildrenAtHome_minmax,TotalChildren_minmax,Age_binned,YearlyIncome_binned,Age_std,YearlyIncome_std,NumberCarsOwned_std,...,StateProvinceName_Tasmania,StateProvinceName_Texas,StateProvinceName_Utah,StateProvinceName_Val d'Oise,StateProvinceName_Val de Marne,StateProvinceName_Victoria,StateProvinceName_Virginia,StateProvinceName_Washington,StateProvinceName_Wyoming,StateProvinceName_Yveline
0,0.183099,0.496842,0.6,0.000000,0.333333,1,3,-0.547868,0.298555,1.892524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.408451,0.489453,0.4,0.333333,0.666667,4,3,0.871862,0.271180,0.798389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.211268,0.536172,0.6,0.000000,0.000000,2,3,-0.370402,0.444261,1.892524,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.323944,0.317083,0.4,0.333333,0.666667,3,2,0.339463,-0.367401,0.798389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.366197,0.231958,0.2,0.000000,0.000000,3,1,0.605663,-0.682765,-0.295746,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Part III: Calculating Proximity /Correlation Analysis of two features
Make sure each attribute is transformed in a same scale for numeric attributes and Binarization for each
nominal attribute, and each discretized numeric attribute to standardization. Make sure to apply a correct
similarity measure for nominal (one hot encoding)/binary attributes and numeric attributes respectively.
(a) Calculate Similarity in Simple Matching, Jaccard Similarity, and Cosine Similarity between two
following objects of your transformed input data.
(b) Calculate Correlation between two features Commute Distance and Yearly Income

In [17]:


from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

obj1 = final_df.iloc[0]
obj2 = final_df.iloc[1]
A = obj1.values
B = obj2.values

# Simple Matching Coefficient
smc = np.sum(A == B) / len(A)
print("SMC:", smc)

# Jaccard Similarity for binary features
jaccard = np.sum((A == 1) & (B == 1)) / np.sum((A == 1) | (B == 1))
print("Jaccard:", jaccard)

# Cosine Similarity
cosine = cosine_similarity([A], [B])[0][0]
print("Cosine:", cosine)


SMC: 0.8157894736842105
Jaccard: 0.3333333333333333
Cosine: 0.6565390577940613
